In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from statistics import mode 
# Input data files are available in the "./input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("./input"))

['Data_Dictionary.xlsx', 'new_merchant_transactions.csv', 'test.csv', 'merchants.csv', 'historical_transactions.csv', 'all.zip', 'train.csv', 'sample_submission.csv']


In [2]:
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")
merchants_df = pd.read_csv("./input/merchants.csv")
new_merchant_transactions_df = pd.read_csv("./input/new_merchant_transactions.csv", )
historical_transactions_df = pd.read_csv("./input/historical_transactions.csv")

In [3]:
def memory_optimized_df(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))\
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')
            
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df        

In [4]:
submission_df = pd.DataFrame({'card_id': test_df["card_id"].values})

In [5]:
merchants_df_ = memory_optimized_df(merchants_df)
new_merchant_transactions_df_ = memory_optimized_df(new_merchant_transactions_df)
historical_transactions_df_ = memory_optimized_df(historical_transactions_df)

del merchants_df
del new_merchant_transactions_df
del historical_transactions_df

Memory usage of dataframe is 56.18 MB
Memory usage after optimization is: 32.66 MB
Decreased by 41.9%
Memory usage of dataframe is 209.67 MB
Memory usage after optimization is: 169.08 MB
Decreased by 19.4%
Memory usage of dataframe is 3109.54 MB
Memory usage after optimization is: 1622.97 MB
Decreased by 47.8%


In [9]:
new_merchant_transactions_df_['category_2'].fillna(1.0, inplace=True)
new_merchant_transactions_df_['category_3'].fillna('A', inplace=True)

historical_transactions_df_['category_2'].fillna(1.0, inplace=True)
historical_transactions_df_['category_3'].fillna('A', inplace=True)

In [10]:
new_merchant_transactions_df_ = new_merchant_transactions_df_.dropna()
historical_transactions_df_al_transactions_df_ = historical_transactions_df_.dropna()

In [11]:
train_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [12]:
test_df.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [14]:
train_df['first_active_month'] = pd.to_datetime(train_df['first_active_month'])

In [15]:
test_df['first_active_month'] = pd.to_datetime(test_df['first_active_month'])

In [16]:
def to_binary(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y': 1, 'N': 0})
    return df

In [18]:
historical_transactions_df_ = to_binary(historical_transactions_df_)
new_merchant_transactions_df_ = to_binary(new_merchant_transactions_df_)

In [20]:
historical_transactions_df_['category_3'] = historical_transactions_df_['category_3'].astype('category').cat.codes
new_merchant_transactions_df_['category_3'] = new_merchant_transactions_df_['category_3'].astype('category').cat.codes


In [22]:
all_transactions = historical_transactions_df_.append(new_merchant_transactions_df_)

In [23]:
del historical_transactions_df_

del new_merchant_transactions_df_

In [25]:
from scipy import stats

def aggregate_transactions(history):
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    agg_func = {
        'authorized_flag': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
        'category_1': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
        'category_2': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
        'category_3': ['sum', 'mean', lambda x: tuple(stats.mode(x)[0])],
        'merchant_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
        'merchant_category_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
        'state_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
        'city_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
        'subsector_id': ['nunique', lambda x: tuple(stats.mode(x)[0])],
        'purchase_amount': ['sum', 'mean', 'max', 'min', 'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'month_lag': [ lambda x: tuple(stats.mode(x)[0])]
    }
    
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace = True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

In [27]:
all_historical_transactions = aggregate_transactions(all_transactions)

/usr/local/Cellar/python3/3.5.1/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [28]:
all_historical_transactions.columns = ['hist_' + c if c != 'card_id' else c for c in all_historical_transactions.columns]

In [29]:
train = pd.merge(train_df, all_historical_transactions, on ='card_id', how='left')
test = pd.merge(test_df, all_historical_transactions, on='card_id', how='left')

In [30]:
train.columns

Index(['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_transactions_count', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_category_1_sum', 'hist_category_1_mean',
       'hist_category_1_<lambda>', 'hist_installments_sum',
       'hist_installments_mean', 'hist_installments_max',
       'hist_installments_min', 'hist_installments_std',
       'hist_city_id_nunique', 'hist_city_id_<lambda>',
       'hist_subsector_id_nunique', 'hist_subsector_id_<lambda>',
       'hist_merchant_id_nunique', 'hist_merchant_id_<lambda>',
       'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_<lambda>', 'hist_authorized_flag_sum',
       'hist_authorized_flag_mean', 'hist_authorized_flag_<lambda>',
       'hist_category_2_sum', 'hist_category_2_mean',
       'hist_category_2_<lambda>', 'hist_state_id_nunique',
       

In [31]:
train['hist_authorized_flag_<lambda>'] = train['hist_authorized_flag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_category_1_<lambda>'] = train['hist_category_1_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [32]:
train['hist_category_2_<lambda>'] = train['hist_category_2_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_category_3_<lambda>'] = train['hist_category_3_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_merchant_id_<lambda>'] = train['hist_merchant_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
train['hist_merchant_category_id_<lambda>'] = train['hist_merchant_category_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_state_id_<lambda>'] = train['hist_state_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_city_id_<lambda>'] = train['hist_city_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_subsector_id_<lambda>'] = train['hist_subsector_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
train['hist_month_lag_<lambda>'] = train['hist_month_lag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [33]:
test['hist_authorized_flag_<lambda>'] = test['hist_authorized_flag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_category_1_<lambda>'] = test['hist_category_1_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_category_2_<lambda>'] = test['hist_category_2_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test['hist_category_3_<lambda>'] = test['hist_category_3_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))

test['hist_merchant_id_<lambda>'] = test['hist_merchant_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)'))
test['hist_merchant_category_id_<lambda>'] = test['hist_merchant_category_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_state_id_<lambda>'] = test['hist_state_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_city_id_<lambda>'] = test['hist_city_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_subsector_id_<lambda>'] = test['hist_subsector_id_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)
test['hist_month_lag_<lambda>'] = test['hist_month_lag_<lambda>'].map(lambda x: str(x).lstrip('(').rstrip(',)')).astype(int)

In [34]:
test.columns = ['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'hist_transactions_count', 'hist_authorized_flag_sum',
       'hist_authorized_flag_mean', 'hist_authorized_flag_mode',
       'hist_category_1_sum', 'hist_category_1_mean',
       'hist_category_1_mode', 'hist_category_2_sum',
       'hist_category_2_mean', 'hist_category_2_mode',
       'hist_category_3_sum', 'hist_category_3_mean',
       'hist_category_3_mode', 'hist_merchant_id_nunique',
       'hist_merchant_id_mode', 'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_mode', 'hist_state_id_nunique',
       'hist_state_id_mode', 'hist_city_id_nunique',
       'hist_city_id_mode', 'hist_subsector_id_nunique',
       'hist_subsector_id_mode', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_installments_sum', 'hist_installments_mean',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_std', 'hist_month_lag_mode']

In [35]:
train.columns = ['first_active_month', 'card_id', 'feature_1', 'feature_2', 'feature_3',
       'target', 'hist_transactions_count', 'hist_authorized_flag_sum',
       'hist_authorized_flag_mean', 'hist_authorized_flag_mode',
       'hist_category_1_sum', 'hist_category_1_mean',
       'hist_category_1_mode', 'hist_category_2_sum',
       'hist_category_2_mean', 'hist_category_2_mode',
       'hist_category_3_sum', 'hist_category_3_mean',
       'hist_category_3_mode', 'hist_merchant_id_nunique',
       'hist_merchant_id_mode', 'hist_merchant_category_id_nunique',
       'hist_merchant_category_id_<lambda>', 'hist_state_id_nunique',
       'hist_state_id_mode', 'hist_city_id_nunique',
       'hist_city_id_mode', 'hist_subsector_id_nunique',
       'hist_subsector_id_mode', 'hist_purchase_amount_sum',
       'hist_purchase_amount_mean', 'hist_purchase_amount_max',
       'hist_purchase_amount_min', 'hist_purchase_amount_std',
       'hist_installments_sum', 'hist_installments_mean',
       'hist_installments_max', 'hist_installments_min',
       'hist_installments_std', 'hist_month_lag_mode']

In [36]:
test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,hist_transactions_count,hist_authorized_flag_sum,hist_authorized_flag_mean,hist_authorized_flag_mode,hist_category_1_sum,...,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_month_lag_mode
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,71,-42.511078,-0.598748,0.235676,-0.743903,...,1,71.0,1.000000,1.0,3,12,-6,94.0,1.323944,1
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,87,-55.080849,-0.633113,0.318817,-0.740723,...,1,335.0,3.849609,4.0,3,13,-10,90.0,1.034483,1
2,2017-08-01,C_ID_b709037bc5,5,1,1,15,4.708528,0.313902,2.525867,-0.724609,...,1,63.0,4.199219,5.0,5,5,-3,22.0,1.466667,2
3,2017-12-01,C_ID_d27d835a9f,2,1,0,36,-19.435101,-0.539864,0.087965,-0.731881,...,1,40.0,1.111328,1.0,3,9,-2,46.0,1.277778,1
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,116,37.205791,0.320740,15.782255,-0.746758,...,1,446.0,3.845703,4.0,4,13,0,122.0,1.051724,1


In [37]:
del all_historical_transactions

In [39]:
merchants_df_.isna().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [41]:
merchants_df_['avg_sales_lag3'].fillna(1.0, inplace = True)
merchants_df_['avg_sales_lag6'].fillna(1.0, inplace = True)
merchants_df_['avg_sales_lag12'].fillna(1.0, inplace = True)

In [42]:
merchants_df_['category_2'].fillna(1.0, inplace = True)

In [44]:
target = train.target

In [47]:
merchants_df_['category_1'] = merchants_df_['category_1'].astype('category').cat.codes
merchants_df_['most_recent_sales_range'] = merchants_df_['most_recent_sales_range'].astype('category').cat.codes
merchants_df_['most_recent_purchases_range'] = merchants_df_['most_recent_purchases_range'].astype('category').cat.codes
merchants_df_['category_4'] = merchants_df_['category_4'].astype('category').cat.codes

In [49]:
merchants_df_.apply(lambda x: len(x.unique()))

merchant_id                    334633
merchant_group_id              109391
merchant_category_id              324
subsector_id                       41
numerical_1                       950
numerical_2                       944
category_1                          2
most_recent_sales_range             5
most_recent_purchases_range         5
avg_sales_lag3                   3372
avg_purchases_lag3             100003
active_months_lag3                  3
avg_sales_lag6                   4507
avg_purchases_lag6             135202
active_months_lag6                  6
avg_sales_lag12                  5009
avg_purchases_lag12            172917
active_months_lag12                12
category_4                          2
city_id                           271
state_id                           25
category_2                          5
dtype: int64

In [50]:
def aggregate_merchants(history):
    
    agg_funcs = {
    
        'merchant_group_id': [lambda x: tuple(stats.mode(x)[0])],
        'merchant_category_id': [lambda x: tuple(stats.mode(x)[0])],
        'subsector_id': [lambda x: tuple(stats.mode(x)[0])],
        'numerical_1': ['sum', 'mean'],
        'numerical_2': ['sum', 'mean'],
        'category_1': [lambda x: tuple(stats.mode(x)[0])],
        'most_recent_sales_range': [lambda x: tuple(stats.mode(x)[0])],
        'most_recent_purchases_range': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag3': ['sum', 'mean'],
        'avg_purchases_lag3': ['sum', 'mean'],
        'active_months_lag3': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag6': ['mean',lambda x: tuple(stats.mode(x)[0])],
        'avg_purchases_lag6': ['sum', 'mean'],
        'active_months_lag6': [lambda x: tuple(stats.mode(x)[0])],
        'avg_sales_lag12': ['sum', 'mean'],
        'avg_purchases_lag12': ['sum', 'mean'],
        'active_months_lag12': [lambda x: tuple(stats.mode(x)[0])],
        'category_4': [lambda x: tuple(stats.mode(x)[0])],
        'city_id': [lambda x: tuple(stats.mode(x)[0])],
        'state_id': [lambda x: tuple(stats.mode(x)[0])],
        'category_2': [lambda x: tuple(stats.mode(x)[0])]
        
    }
    
    agg_history = history.groupby(['merchant_id']).agg(agg_funcs)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('merchant_id')
          .size()
          .reset_index(name='merchant_count'))
    
    agg_history = pd.merge(df, agg_history, on='merchant_id', how='left')
    
    return agg_history

In [ ]:
merchants_data_all = aggregate_merchants(merchants_df_)